#### Building  a character-level model Seq2Seq Language Translation model for English- Hindi 

#### Import Libraries

In [1]:
from __future__ import print_function
import string
import matplotlib.pyplot as plt
from string import digits
%matplotlib inline
import re
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model

#### Mount Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Load and Preprocess Data

In [3]:
batch_size = 64
epochs = 250 
latent_dim = 256 
num_samples = 38000 
data_path = 'drive/My Drive/hin.txt'

In [4]:
exclude = set(string.punctuation)
remove_digits = str.maketrans('', '', digits)
Eng_texts = []
Hin_texts = []
Eng_characters = set()
Hin_characters = set()

with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    Eng_text, Hin_text, _ = line.split('\t')
    Eng_text = Eng_text.lower()
    Hin_text = Hin_text.lower()
    Eng_text = re.sub("'", '', Eng_text)
    Eng_text = re.sub(",", ' COMMA', Eng_text)
    Hin_text = re.sub("'", '', Hin_text)
    Hin_text = re.sub(",", ' COMMA', Hin_text)
    Eng_text = ''.join(x for x in Eng_text if x not in exclude)
    Hin_text = ''.join(x for x in Hin_text if x not in exclude)
    Eng_text = Eng_text.translate(remove_digits)
    Hin_text = Hin_text.translate(remove_digits)
    Hin_text = '\t' + Hin_text + '\n'
    Eng_texts.append(Eng_text)
    Hin_texts.append(Hin_text)
    for char in Eng_text:
        if char not in Eng_characters:
            Eng_characters.add(char)
    for char in Hin_text:
        if char not in Hin_characters:
            Hin_characters.add(char)

In [6]:
Eng_characters = sorted(list(Eng_characters))
Hin_characters = sorted(list(Hin_characters))
num_encoder_tokens = len(Eng_characters)
num_decoder_tokens = len(Hin_characters)
max_encoder_seq_length = max([len(txt) for txt in Eng_texts])
max_decoder_seq_length = max([len(txt) for txt in Hin_texts])

print('Number of samples:', len(Eng_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 2773
Number of unique input tokens: 31
Number of unique output tokens: 82
Max sequence length for inputs: 109
Max sequence length for outputs: 133


#### Create a dictionary for english and hindi characters

In [7]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(Eng_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(Hin_characters)])


encoder_input_data = np.zeros(
    (len(Eng_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(Eng_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(Eng_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

#### One Hot Encoding

In [8]:
for i, (Eng_text, Hin_text) in enumerate(zip(Eng_texts, Hin_texts)):
    for t, char in enumerate(Eng_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(Hin_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

#### Create Model 

In [9]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True, dropout = 0.3)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, dropout=0.3)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

#### Compile  and Fit  Model 

In [10]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.1)

Epoch 1/250
39/39 [==============================] - 43s 1s/step - loss: 1.0886 - accuracy: 0.7941 - val_loss: 1.6631 - val_accuracy: 0.6739
Epoch 2/250
39/39 [==============================] - 42s 1s/step - loss: 0.8264 - accuracy: 0.8163 - val_loss: 1.5676 - val_accuracy: 0.6744
Epoch 3/250
39/39 [==============================] - 42s 1s/step - loss: 0.7908 - accuracy: 0.8179 - val_loss: 1.4155 - val_accuracy: 0.6764
Epoch 4/250
39/39 [==============================] - 42s 1s/step - loss: 0.7442 - accuracy: 0.8207 - val_loss: 1.3730 - val_accuracy: 0.6781
Epoch 5/250
39/39 [==============================] - 42s 1s/step - loss: 0.6870 - accuracy: 0.8293 - val_loss: 1.2475 - val_accuracy: 0.6869
Epoch 6/250
39/39 [==============================] - 42s 1s/step - loss: 0.6994 - accuracy: 0.8329 - val_loss: 1.1926 - val_accuracy: 0.6956
Epoch 7/250
39/39 [==============================] - 42s 1s/step - loss: 0.6238 - accuracy: 0.8398 - val_loss: 1.0969 - val_accuracy: 0.7255
Epoch 8/250
3

In [11]:
# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

#### Predictions

In [12]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

#### Inference Samples

In [15]:
for seq_index in range(2011,2017):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)


    
    print('Input sentence:', Eng_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

        


Input sentence: we talked to each other for a while
Decoded sentence: हमे पिछने में की एक खड़ी थी।

Input sentence: we were all very happy at breakfast
Decoded sentence: हम सब तुम्हारे साथ एक रहन करने पड़ेगी।

Input sentence: what are you going to do in college
Decoded sentence: तुम आज राज़ी को खाल आना खाले के लिए क्या करते हो

Input sentence: what do you usually do after dinner
Decoded sentence: तुम आज रात सा खोलो।

Input sentence: what was the weather like yesterday
Decoded sentence: इस कितने का समय मताब लम्बी हैं

Input sentence: what would the world do without tea
Decoded sentence: तुम कहाँ जाते हो

